In [ ]:
class Training_Data:
    
    def __init__(self, data_file):
        self.data_file = data_file
        
    def load_data(self):
        
        import os
    
        file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', self.data_file))    
        with open(file_path, 'rt', encoding='utf-8') as tf:
            loaded_ad = tf.read().split('\n')
        job_ads = []
        job_ads = [self.parse_data(item) for item in loaded_ad[:-1]]
        self.raw_data = job_ads
        self.classes = set([item['class'] for item in job_ads])
    
    def parse_data(self, input_line):
        
        cur_text = input_line.split('`')
        ret_data = {}
        ret_data['id'] = cur_text[1]
        ret_data['company'] = cur_text[3]
        ret_data['position'] = cur_text[5]
        ret_data['url'] = cur_text[7]
        ret_data['desc'] = cur_text[9]
        ret_data['class'] = ''.join(cur_text[11:]).replace(',NA', '')
    
        return ret_data
    
    def create_training_set(self, labels):
        
        def remove_white_space(in_text):
            
            on_text = in_text[:].replace(u'\xa0', ' ')
            while on_text.find('  ') > -1:
                on_text = on_text.replace('  ', ' ')
                
            return on_text
        
        self.label_dict = {}
        for i, item in enumerate(labels):
            self.label_dict[item] = i + 1
        self.sample_desc = [remove_white_space(item['desc']) for item in self.raw_data]
        self.sample_title = [remove_white_space(item['position']) for item in self.raw_data]
        self.label = [self.label_dict[item['class']] if item['class'] in self.label_dict else 0 for item in self.raw_data]
        
    def balance_training_set(self, balance_class):
    
        import numpy as np
        prop = np.mean([np.array(self.label) == self.label_dict[balance_class]])
        prop = prop / (1 - prop)
        
        mask = [True if item == self.label_dict[balance_class] or np.random.rand() < prop else False for item in self.label]
        sample_desc = []
        sample_title = []
        label = []
        for i, item in enumerate(mask):
            if item:
                sample_desc.append(self.sample_desc[i])
                sample_title.append(self.sample_title[i])
                label.append(self.label[i])
        self.sample_desc = sample_desc
        self.sample_title = sample_title
        self.label = label

In [ ]:
TD = Training_Data('training_set_20170313.csv')
TD.load_data()
TD.create_training_set(['OTHER-Clearks, secretary, and general office workers'])
TD.balance_training_set('OTHER-Clearks, secretary, and general office workers')